In [2]:
# https://ithelp.ithome.com.tw/articles/10321898
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# 設定爬取的URL
def allUrl(loopTimes, urlBase):
    for i in range(loopTimes):
        url = urlBase + str(i+1) + '&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1'
        print(url)
        # 發送GET請求並取得響應
        response = requests.get(url)

        # 檢查響應狀態碼，確保成功連接到網站
        if response.status_code == 200:
            # 解析HTML頁面
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 獲取職缺列表
            job_list = soup.find_all('article', class_='js-job-item')
            
            for job in job_list:
                
                jobLink = re.findall(r'<a class="js-job-link" data-qa-id="jobSeachResultTitle" href="([^"]*)', job.prettify(), flags=re.DOTALL)
                # jobLink=jobLink.str.split('/').str[1]
                
                # 提取職缺標題
                title = job.find('a', class_='js-job-link').text.strip()
                
                # 提取公司名稱
                company = job.get('data-cust-name').strip()

                raw=pd.DataFrame({'url':jobLink, 'title':title, 'company': company})
                raw.to_csv('allLink.csv',mode='a', index=0, encoding='utf-8-sig')
                
                # # 提取工作地點
                # location = job.find('ul', class_='job-list-intro').find_all('li')[0].text.strip()
                
                # # 提取薪資
                # salary = job.find('span', class_='b-tag--default').text.strip()

        else:
            print("連接網站失敗，狀態碼：", response.status_code)
            
# 搜尋另一個關鍵字(Power BI)
allUrl(9, 'https://www.104.com.tw/jobs/search/?ro=0&keyword=Power%20BI%E2%80%8B&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=')

# 搜尋另一個關鍵字(vba)
allUrl(9, 'https://www.104.com.tw/jobs/search/?ro=0&keyword=vba&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=')

# 搜尋另一個關鍵字(python)
allUrl(9, 'https://www.104.com.tw/jobs/search/?ro=0&keyword=python&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=')

# 網址是我篩選好要的條件與關鍵字(資料分析)，迴圈9次，抓取9頁
allUrl(9, 'https://www.104.com.tw/jobs/search/?ro=0&keyword=%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=')

# 搜尋另一個關鍵字(sql)
allUrl(9, 'https://www.104.com.tw/jobs/search/?ro=0&keyword=sql&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=')

# 搜尋另一個關鍵字(ga)
allUrl(9, 'https://www.104.com.tw/jobs/search/?ro=0&keyword=ga&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=')

# 讀取抓到的所有網址，並篩選出工作職缺的網址
raw=pd.read_csv('allLink.csv')
df = raw.copy()
df=df.drop_duplicates()
df = df[df['url'] != 'url']
df['url'] = df['url'].str.split('//').str[1]
df = df[~df['url'].str.contains('%')]
df = df[~df['url'].str.contains('blackbar')]
df = df[~df['url'].str.contains('company')].reset_index(drop=1)

# 排除職缺關鍵字
df = df[~df['title'].str.contains(
'後端|前端|FAE|軟體|資安|主管|課長|小編|全端|主任|業務|講師|SEO|網路管理|網管|會計|資深|高級')]

# 讀取抓到的所有網址，並篩選出工作職缺的網址
raw=pd.read_csv('allLink.csv')
df = raw.copy()
df=df.drop_duplicates()
df = df[df['url'] != 'url']
df['url'] = df['url'].str.split('//').str[1]
df = df[~df['url'].str.contains('%')]
df = df[~df['url'].str.contains('blackbar')]
df = df[~df['url'].str.contains('company')].reset_index(drop=1)

# 讀取我已經看過的網址並排除
exclusive=pd.read_csv('exclusive.csv',on_bad_lines='skip', encoding='big5hkscs')
merge = pd.merge(df,exclusive[['url','reason']],on='url',how='left')
merge = merge[merge['reason'].isna()]

merge['concat'] = merge['title'] + merge['company']
df['concat'] = df['title'] + df['company']
merge = pd.merge(merge,df['concat'],on='concat',how='left')
merge = merge[merge['reason'].isna()]

merge=merge.sort_values(by='reason')
merge=merge.drop_duplicates(subset='url').reset_index(drop=1)
merge = merge[['url', 'reason', 'title', 'company']]
merge.to_csv('link.csv', index=0, encoding='utf-8-sig')
merge

https://www.104.com.tw/jobs/search/?ro=0&keyword=Power%20BI%E2%80%8B&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1
https://www.104.com.tw/jobs/search/?ro=0&keyword=Power%20BI%E2%80%8B&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=2&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1
https://www.104.com.tw/jobs/search/?ro=0&keyword=Power%20BI%E2%80%8B&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001008005%2C6001008006%2C6001008004%2C6001008002%2C6001008003%2C6001008001%2C6001008007&edu=1%2C2%2C3%2C4&order=14&asc=0&s5=0&page=3&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&h

,url,reason,title,company
0,www.104.com.tw/job/84xxj?jobsource=hotjob_chr,NaN,軟體測試工程師,何茂貿易有限公司
1,www.104.com.tw/job/7rej5?jobsource=2018indexpoc,NaN,資訊部門-資訊服務管理人員,台中商業銀行股份有限公司
2,www.104.com.tw/job/7ck27?jobsource=2018indexpoc,NaN,財務會計,宙隆企業股份有限公司
3,www.104.com.tw/job/67cmm?jobsource=2018indexpoc,NaN,品牌企劃專員,有物設計有限公司
4,www.104.com.tw/job/83fba?jobsource=2018indexpoc,NaN,【品牌部】品牌數位行銷經理 Digital Marketing Manager,潤室股份有限公司
...,...,...,...,...
612,www.104.com.tw/job/45o7b?jobsource=2018indexpoc,NaN,International Sales Representative (Internatio...,大阪國際實業有限公司
613,www.104.com.tw/job/7w8a0?jobsource=2018indexpoc,NaN,【行銷部】品牌行銷企劃專員／品牌策略管理師,羽軒股份有限公司
614,www.104.com.tw/job/7c26e?jobsource=2018indexpoc,NaN,品牌及電商行銷部｜專員,台灣碳資產股份有限公司
615,www.104.com.tw/job/7qr96?jobsource=2018indexpoc,NaN,系統專案管理師(PM),華瑞資訊股份有限公司
